In [1]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import numpy as np
import pandas as pd
import isodate
import requests
import time
from selenium import webdriver
from IPython.display import JSON

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
# api key load
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [4]:
# URL 모음
base_url = "https://store.steampowered.com/"

url_stats = 'stats/'
url_search = 'search/'
url_category = 'category/'

#url_filter = '?filter='
url_filter = '?flavor=contenthub_newandtrending'

# offset
url_offset = '&offset='

# 스팀 장르별 URL 모음
url_action = 'action/'
url_action_fps = 'action_fps/'
url_action_tps = 'action_tps/'
url_fighting_marial_arts = 'fighting_marial_arts/'
url_shump = 'shump/'
url_arcade_rhythm = 'arcade_rhythm/'
url_action_run_jump = 'action_run_jump/'
url_hack_and_slash = 'hack_and_slash/'

url_rpg = 'rpg/'
url_jrpg = 'rpg_jrpg/'
url_rogue_like_rogue_lite = 'rogue_like_rogue_lite/'
url_rpg_action = 'rpg_action/'
url_adventure_rpg = 'adventure_rpg/'
url_rpg_strategy = 'rpg_strategy_tactics/'
url_rpg_turn_based = 'rpg_turn_based/'
url_rpg_party_based = 'rpg_party_based/'

url_strategy = 'strategy/'
url_strategy_military = 'strategy_military/'
url_strategy_grand_4x = 'strategy_grand_4x/'
url_strategy_cities_settlements = 'strategy_cities_settlements/'
url_strategy_real_time = 'strategy_real_time/'
url_strategy_card_board = 'strategy_card_board/'
url_tower_defense = 'tower_defense/'
url_strategy_turn_based = 'strategy_turn_based/'

url_adventure = 'adventure/'
url_metroidvania = 'metroidvania/'
url_visual_novel = 'visual_novel/'
url_adventure_rpg = 'adventure_rpg/'
url_casual = 'casual/'
url_puzzle_matching = 'puzzle_matching/'
url_story_rich = 'story_rich/'
url_hidden_object = 'hidden_object/'

rul_simulation = 'simulation/'
rul_sim_building_automation = 'sim_building_automation/'
rul_sim_farming_crafting = 'sim_farming_crafting/'
rul_sim_physics_sandbox = 'sim_physics_sandbox/'
rul_sim_life = 'sim_life/'
rul_sim_dating = 'sim_dating/'
rul_sim_space_flight = 'sim_space_flight/'
rul_sim_hobby_sim = 'sim_hobby_sim/'

url_sports_and_racing = 'sports_and_racing/'
url_sports_individual = 'sports_individual/'
url_sports_fishing_hunting = 'sports_fishing_hunting/'
url_racing = 'racing/'
url_racing_sim = 'racing_sim/'
url_sports = 'sports/'
url_sports_sim = 'sports_sim/'
url_sports_team = 'sports_team/'

## 테마 별 URL
url_science_fiction = 'science_fiction/'
url_horror = 'horror/'
url_mystery_detective = 'mystery_detective/'
url_survival = 'survival/'
url_adultonly = 'adultonly/'
url_anime = 'anime/'
url_exploration_open_world = 'exploration_open_world/'
url_space = 'space/'

## 플레이어 지원 별 URL
url_multiplayer_lan = 'multiplayer_lan/'
url_multiplayer_mmo = 'multiplayer_mmo/'
url_multiplayer_local_party = 'multiplayer_local_party/'
url_multiplayer = 'multiplayer/'
url_singleplayer = 'singleplayer/'
url_multiplayer_online_competitive = 'multiplayer_online_competitive/'
url_multiplayer_coop = 'multiplayer_coop/'

In [6]:
## URL 합치기
use_url = base_url + url_category + url_action + url_filter

In [7]:
def new_popular_game_scan(use_URL):
    # Selenium 옵션 설정
    options = Options()
    options.headless = True

    # Chrome Driver 실행
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    driver.get(use_URL)
    
    game_data = []
    time.sleep(10)
    games = driver.find_elements(By.CLASS_NAME, "_1lRFu670LVk6Gmeb12h7Hr")
    time.sleep(10)
    
    # 게임 정보 리스트에 추가
    for game in games:
        name = game.find_element(By.CLASS_NAME, "_3jI467XYJLy1CQ5YZhp2q_").text.strip()
        time.sleep(5)
        price = game.find_element(By.CLASS_NAME, "Wh0L8EnwsPV_8VAu8TOYr").text.strip()
        release_date = game.find_element(By.CLASS_NAME, "_3eOdkTDYdWyo_U5-JPeer1").text.strip()
        tags = game.find_elements(By.CLASS_NAME, "WidgetTag")

        tag_list = [tag.text for tag in tags]
        tag_list += [''] * (5 - len(tag_list))

        # 게임 데이터에 추가
        game_data.append([name, price, release_date] + tag_list[:5])

    # 데이터프레임 생성
    new_games = pd.DataFrame(game_data, columns=['게임 이름', '가격', '출시일', 'Tag 1', 'Tag 2', 'Tag 3', 'Tag 4', 'Tag 5'])
    driver.quit()
    
    return new_games

In [9]:
df_list = []

for i in range(0, 5):
    offset = str(12 * i)
    use_url = base_url + url_category + url_action + url_filter + url_offset + offset
    
    temp = new_popular_game_scan(use_url)
    df_list.append(temp)
    time.sleep(10)
    
df_combined = pd.concat(df_list, ignore_index=True)
df_combined

,게임 이름,가격,출시일,Tag 1,Tag 2,Tag 3,Tag 4,Tag 5
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,Command & Conquer Red Alert™ 2 and Yuri’s Reve...,"₩ 14,200","MAR 8, 2024",Action,Strategy,RPG,RTS,Singleplayer
...,...,...,...,...,...,...,...,...
74,Banishers: Ghosts of New Eden,"₩ 58,000","FEB 13, 2024",Narration,Lore-Rich,Third Person,Emotional,Atmospheric
75,The Weird Dream,"₩ 13,170","MAR 10, 2024",Adventure,Action,Metroidvania,Action-Adventure,2D Platformer
76,Uootoy 3671,"₩ 14,020","MAR 14, 2024",Action,Casual,Puzzle,Puzzle Platformer,2D
77,Sigils,"₩ 11,000","MAR 15, 2024",Action,Strategy,Card Game,Magic,Dragons


In [10]:
df3 = df_combined.replace('', np.nan)
data = df3.dropna()
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임 이름   60 non-null     object
 1   가격      60 non-null     object
 2   출시일     60 non-null     object
 3   Tag 1   60 non-null     object
 4   Tag 2   60 non-null     object
 5   Tag 3   60 non-null     object
 6   Tag 4   60 non-null     object
 7   Tag 5   60 non-null     object
dtypes: object(8)
memory usage: 3.9+ KB


In [11]:
data

,게임 이름,가격,출시일,Tag 1,Tag 2,Tag 3,Tag 4,Tag 5
0,Command & Conquer Red Alert™ 2 and Yuri’s Reve...,"₩ 14,200","MAR 8, 2024",Action,Strategy,RPG,RTS,Singleplayer
1,Command & Conquer™ Generals Zero Hour,"₩ 14,200","MAR 8, 2024",Strategy,Action,Singleplayer,RTS,RPG
2,HELLDIVERS™ 2,"₩ 44,800","FEB 8, 2024",Online Co-Op,Action,Multiplayer,Third-Person Shooter,Co-op
3,A Difficult Game About Climbing,"₩ 11,000","MAR 7, 2024",Difficult,Precision Platformer,Psychological Horror,Physics,Funny
4,Contra: Operation Galuga,"₩ 51,390","MAR 12, 2024",Action,Arcade,Shooter,Platformer,2D Platformer
5,Command & Conquer™ Generals,"₩ 14,200","MAR 8, 2024",Strategy,Action,Singleplayer,RTS,Action RPG
6,TEKKEN 8,"₩ 79,800","JAN 26, 2024",Fighting,Action,3D Fighter,Multiplayer,eSports
7,Last Epoch,"₩ 37,500","FEB 22, 2024",Action RPG,Loot,Hack and Slash,RPG,Isometric
8,The Outlast Trials,"₩ 42,000","MAR 5, 2024",Horror,Multiplayer,Co-op,Survival Horror,Psychological Horror
9,Granblue Fantasy: Relink,"₩ 69,800","FEB 1, 2024",Anime,Action,Action RPG,JRPG,Online Co-Op


In [12]:
data.to_csv('Action new_popular_games.csv', encoding="utf-8-sig", index=False)

In [16]:
tags_columns = ['Tag 1', 'Tag 2', 'Tag 3', 'Tag 4', 'Tag 5']
all_tags = pd.Series(data[tags_columns].values.flatten())
most_common_tags = all_tags.value_counts()
most_common_tags[1:6]

Singleplayer    12
Strategy        10
RPG              9
Action RPG       8
RTS              7
dtype: int64

In [17]:
game_names = data['게임 이름'].tolist()
game_names

['Command & Conquer Red Alert™ 2 and Yuri’s Revenge™',
 'Command & Conquer™ Generals Zero Hour',
 'HELLDIVERS™ 2',
 'A Difficult Game About Climbing',
 'Contra: Operation Galuga',
 'Command & Conquer™ Generals',
 'TEKKEN 8',
 'Last Epoch',
 'The Outlast Trials',
 'Granblue Fantasy: Relink',
 'Command & Conquer™ Tiberian Sun™ and Firestorm™',
 'TRIBES 3: Rivals',
 'Command & Conquer Renegade™',
 'The Saboteur™',
 'Deep Rock Galactic: Survivor',
 'Like a Dragon: Infinite Wealth',
 'Command & Conquer™ and The Covert Operations™',
 'Command & Conquer Red Alert™, Counterstrike™ and The Aftermath™',
 'BORE BLASTERS',
 'Ys X: Nordics',
 'Pacific Drive',
 'Toy Shire: Room One',
 'BlazBlue Entropy Effect',
 'Highwater',
 'Palworld',
 'STAR WARS™: Battlefront Classic Collection',
 'Touhou Danmaku Kagura Phantasia Lost',
 '迷宮校舎 | School Labyrinth',
 'Tomb Raider I-III Remastered Starring Lara Croft',
 'Rebel Transmute',
 'Suicide Squad: Kill the Justice League',
 'Wo Long: Fallen Dynasty',
 'Duel

In [18]:
def search_new_games(game_names):
    video_results = []
    for game_name in game_names:
        request = youtube.search().list(
            q=game_name,
            part='id,snippet',
            maxResults=5,
            type='video'
        )
        response = request.execute()
        videos = response['items']
        for video in videos:
            video_id = video['id']['videoId']
            video_request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=video_id
            )
            video_response = video_request.execute()
            video_info = video_response['items'][0]
            video_results.append({
                'Game': game_name,
                'Title': video_info['snippet']['title'],
                'Video ID': video_id,
                'Channel': video_info['snippet']['channelTitle'],
                'Duration': video_info['contentDetails']['duration'],
                'Definition': video_info['contentDetails']['definition'],
                'Caption': video_info['contentDetails'].get('caption', ''),
                'View Count': video_info['statistics']['viewCount'],
                'Like Count': video_info['statistics']['likeCount'],
                'Favorite Count': video_info['statistics']['favoriteCount'],
                'Comment Count': video_info['statistics']['commentCount']
            })

    # 데이터프레임으로 변환
    video_df = pd.DataFrame(video_results)
    return video_df

In [19]:
def change_time(df, col):
    df[col] = df[col].apply(lambda x: isodate.parse_duration(x).total_seconds())

    return df

In [21]:
new_game_videos = search_new_games(game_names[:30])